In [33]:
import itertools, collections
from tabulate import tabulate
import numpy as np
from typing import List, Tuple, Dict
import sympy
#init_printing()
my_vars = set(dir())
sympy.init_printing()

In [34]:
y, a, b, c, d, e, f, g = sympy.symbols('y a b c d e f g')
y = a + b

In [35]:
y

In [36]:
class thresholdImplement:
    def __init__(self, num_shares: int, orig_bool) -> None:
        self.num_shares     = num_shares
        self.orig_bool      = orig_bool
        self.i_vars_str     = sorted([x.name for x in orig_bool.free_symbols])
        self.i_vars         = []
        # Due to abitary order of python set variable (orig_bool.free_symbols)
        for i_var_str in self.i_vars_str:
            for i_var in list(orig_bool.free_symbols):
                if i_var_str == i_var.name:
                    self.i_vars.append(i_var)
        self.i_vars_num     = len(self.i_vars)
        self.i_share_vars   = []
        self.i_share_val0   = []
        self.i_share_val1   = []
        self.o_share_vars   = []
        self.io_shares_dict = collections.OrderedDict()
        self.io_counts_dict = collections.OrderedDict()
        self.gen_o_symbols()
        self.gen_i_symbols()
        self.find_i_shares()

    def gen_i_symbols(self):
        for i in range(self.i_vars_num):
            for j in range(self.num_shares):
                exec(f"{self.i_vars[i]}_{j+1} = sympy.symbols('{self.i_vars[i]}_{j+1}')")
                exec(f"self.i_share_vars.append({self.i_vars[i]}_{j+1})")
        globals().update(locals())

    def gen_o_symbols(self):
        for i in range(self.num_shares):
            exec(f"y_{i+1} = sympy.symbols('y_{i+1}')")
            exec(f"self.o_share_vars.append(y_{i+1})")
        globals().update(locals())

    def find_i_shares(self):
        for i in range(2**self.num_shares):
            i_share_bits_str = list(bin(i)[2:].zfill(self.num_shares))
            i_share_bits_int = list(map(int, i_share_bits_str))
            if(np.bitwise_xor.reduce(i_share_bits_int) == 0):
                self.i_share_val0.append(i_share_bits_int)
            else:
                self.i_share_val1.append(i_share_bits_int)

    def check_constraints(self, o_share_vars):
        # assign global symbol to local symbol of class
        for idx, o_share_var in enumerate(o_share_vars):
            self.o_share_vars[idx] = o_share_var

        # generate io_shares_dict for all possible cases
        for i in range(2**self.i_vars_num):
            i_bits_str = bin(i)[2:].zfill(self.i_vars_num)
            i_bits_int = list(map(int, i_bits_str))
            # find correct output
            o_correct = self.orig_bool
            for idx, i_var in enumerate(self.i_vars):
                o_correct = o_correct.subs(i_var, i_bits_int[idx])
            self.io_shares_dict[i_bits_str] = []
            # find available patterns of input shares
            i_avail_shares = [self.i_share_val0 if x == '0' else self.i_share_val1 for x in i_bits_str]
            i_avail_shares_combs = list(itertools.product(*i_avail_shares))
            # calculate ouput share value
            for i_avail_shares_comb in i_avail_shares_combs:
                i_avail_shares_vals = list(itertools.chain.from_iterable(i_avail_shares_comb))
                o_share_vals = []
                for o_share_var in self.o_share_vars:
                    for idx, i_share_var in enumerate(self.i_share_vars):
                        o_share_var = o_share_var.subs(i_share_var, i_avail_shares_vals[idx])
                    o_share_vals.append(o_share_var%2)
                # check correctness
                o_shares_xor = np.bitwise_xor.reduce(o_share_vals)
                if o_correct%2 != o_shares_xor:
                    print(f'Correctness Error: Output {o_correct%2} != {o_shares_xor}')
                self.io_shares_dict[i_bits_str].append([''.join(str(x) for x in i_avail_shares_vals), ''.join(str(x) for x in o_share_vals)])
            # count output share cases
            self.io_counts_dict[i_bits_str] = []
            for j in range(2**self.num_shares):
                o_bits_str = bin(j)[2:].zfill(self.num_shares)
                o_share_case_count = list(itertools.chain.from_iterable(list(self.io_shares_dict[i_bits_str]))).count(o_bits_str)
                self.io_counts_dict[i_bits_str].append(o_share_case_count)
        # draw uniformity table
        print(tabulate(list(ti.io_counts_dict.values()), headers=[bin(x)[2:].zfill(self.num_shares) for x in range(2**self.num_shares)]))


In [37]:
ti = thresholdImplement(3, y)
ti.orig_bool

In [38]:
y_1 = a_2*b_2 + a_2*b_3 + a_3*b_2
y_2 = a_3*b_3 + a_1*b_3 + a_3*b_1
y_3 = a_1*b_1 + a_1*b_2 + a_2*b_1

In [42]:
y_1, y_2, y_3

In [23]:
#y_1 = a_1 + b_1 #+ c_1
#y_2 = a_2 + b_2 #+ c_2
#y_3 = a_3 + b_3 #+ c_3

In [24]:
ti.check_constraints([y_1, y_2, y_3])

  000    001    010    011    100    101    110    111
-----  -----  -----  -----  -----  -----  -----  -----
    4      0      0      4      0      4      4      0
    0      4      4      0      4      0      0      4
    0      4      4      0      4      0      0      4
    4      0      0      4      0      4      4      0


In [9]:
ti.o_share_vars[0]

a_1 + b_1

In [25]:
list(ti.io_shares_dict.items())

[('00',
  [['000000', '000'],
   ['000011', '011'],
   ['000101', '101'],
   ['000110', '110'],
   ['011000', '011'],
   ['011011', '000'],
   ['011101', '110'],
   ['011110', '101'],
   ['101000', '101'],
   ['101011', '110'],
   ['101101', '000'],
   ['101110', '011'],
   ['110000', '110'],
   ['110011', '101'],
   ['110101', '011'],
   ['110110', '000']]),
 ('01',
  [['000001', '001'],
   ['000010', '010'],
   ['000100', '100'],
   ['000111', '111'],
   ['011001', '010'],
   ['011010', '001'],
   ['011100', '111'],
   ['011111', '100'],
   ['101001', '100'],
   ['101010', '111'],
   ['101100', '001'],
   ['101111', '010'],
   ['110001', '111'],
   ['110010', '100'],
   ['110100', '010'],
   ['110111', '001']]),
 ('10',
  [['001000', '001'],
   ['001011', '010'],
   ['001101', '100'],
   ['001110', '111'],
   ['010000', '010'],
   ['010011', '001'],
   ['010101', '111'],
   ['010110', '100'],
   ['100000', '100'],
   ['100011', '111'],
   ['100101', '001'],
   ['100110', '010'],
   [

In [10]:
list(ti.io_counts_dict.items())

[('00', [4, 0, 0, 4, 0, 4, 4, 0]),
 ('01', [0, 4, 4, 0, 4, 0, 0, 4]),
 ('10', [0, 4, 4, 0, 4, 0, 0, 4]),
 ('11', [4, 0, 0, 4, 0, 4, 4, 0])]

In [11]:
ti.o_share_vars

[a_1 + b_1, a_2 + b_2, a_3 + b_3]